In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, datetime

## Script Objectives
 - Read in baseline CLN file
 - Read in data regarding CLN's we want to add
 - Add CLN's to baseline CLN file
 - Activate CLN's in IBOUND sections where appropriate
 - Edit IBOUND & STRT for CLN files to account for additional CLN's being included to this particular model simulation

### Sections of CLN input
Sublists are parameters that need to be updated when a new CLN file is written

1. NCLN ICLNNDS ICLNCB ICLNHD ICLNDD ICLNIB NCLNGWC NCONDUITYP
    1. NCLN: number of CLN segments
    2. ICLNNDS: flag that is set to negative NCLN
    3. NCLNGWC: number of GW node connections
   
   
   
2. IFNO IFTYP IFDIR FLENG FELEV FANGLE IFLIN ICCWADI
    a line for each need to be added for every new CLN
    1. FLENG: length of CLN node (sum of thicknesses)
    2. FELEV: bottom elevation of CLN node 
    
    
    
3. IFNOD IGWNOD IFCON FSKIN FLENGW FANISO ICGWADI
    for every new CLN
    1. IFNOD: CLN number
    2. IGWNOD: GW node number CLN is connected to
    3. IFCON: [1] -> Thiem equation used to provide connection betweeen CLN node & GW node, with skin effects
    4. IFSKIN: [16000] skin factor 
    5. FLENGW: length of CLN segment connected to cell (thickness of that layer/cell)
    6. FANISO: [1] used in computation of leakance
    7. ICGWADI [0] cell is treated without vertical flow correction
    

In [2]:
scenario = 'test'

In [3]:
CLN_data = pd.read_excel('input/scenario_inputs.xlsx', sheet_name='CLN_Data', index_col='CLN')
fraction_data = pd.read_excel('input/scenario_inputs.xlsx', sheet_name='Fraction_Active_Data')
new_CLN_data = CLN_data[CLN_data.screen=='s']
new_CLNs = new_CLN_data.index.drop_duplicates().values
print(new_CLNs)

[1793 1794 1795]


C:\Miniconda3\envs\geo_py37\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Miniconda3\envs\geo_py37\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [4]:
fraction_data

,CLN,name,group,type,fraction
0,alamitos,alamitos,Alamitos Barrier,Injection,1.00
1,1793,CB-SW-1,LVL,Injection,0.33
2,1794,CB-SW-2,LVL,Injection,0.33
3,1795,CB-SW-3,LVL,Injection,0.34


### Read and open CLN files to start writing: establish initial data group 1 parameters

In [5]:
base_CLN = open("S:/LAX/DUDEK.C001.SMBGSA/INTERA models/UpdatedModel/ModelWithNewExe/input/lab_ug_Updated.cln", "r")
new_CLN  = open("S:/LAX/JACOBS.C001.CNSLT/Phase-2/LVL/4000.MODELS/test/lab_ug_test.cln", "w")
new_CLN.write(base_CLN.readline())
second_line = base_CLN.readline().split()
base_NCLN, base_NCLNGWC = int(second_line[0]), int(second_line[6])
NCONDUITYP = int(second_line[7])
new_NCLN = base_NCLN + len(new_CLNs)
new_NCLNGWC = base_NCLNGWC + len(new_CLN_data)
new_CLN.write("%d %d 4 199 0 8 %d 11\n" % (new_NCLN, -new_NCLN, new_NCLNGWC))
new_CLN.write(base_CLN.readline())

54

### Write data for data group 2 and write additional lines for new CLN's



In [6]:
for n in range(base_NCLN):
    new_CLN.write(base_CLN.readline()) #write for all existing CLN's

for n in new_CLNs: #for additional CLN's
    FLENG = new_CLN_data.loc[n, "thk"].sum()
    FELEV = (new_CLN_data.loc[n, "surface_elevation"].drop_duplicates() - new_CLN_data.loc[n,"botm_depth"].max()).values[0]
    well_name = new_CLN_data.loc[n,"name"]
    new_CLN.write(
            "%d %d %d %.2lf %.2lf 0 0 0 # %s\n" % (n, 7, 0, FLENG, FELEV, well_name))
    print("%d %d %d %.2lf %.2lf 0 0 0 # %s\n" % (n, 7, 0, FLENG, FELEV, well_name))

1793 7 0 1316.75 -1491.82 0 0 0 # CLN
1793    CB-SW-1
1793    CB-SW-1
1793    CB-SW-1
1793    CB-SW-1
1793    CB-SW-1
Name: name, dtype: object

1794 7 0 1259.57 -1438.61 0 0 0 # CLN
1794    CB-SW-2
1794    CB-SW-2
1794    CB-SW-2
1794    CB-SW-2
1794    CB-SW-2
Name: name, dtype: object

1795 7 0 1404.48 -1591.04 0 0 0 # CLN
1795    CB-SW-3
1795    CB-SW-3
1795    CB-SW-3
1795    CB-SW-3
1795    CB-SW-3
1795    CB-SW-3
Name: name, dtype: object



In [16]:
well_name




'CB-SW-3'

### Write data for data group 3 regading GW node connections

In [8]:
for n in range(base_NCLNGWC):
    new_CLN.write(base_CLN.readline()) #write for all existing CLN's
    
for n in new_CLNs: 
    well_name = new_CLN_data.loc[n,"name"].drop_duplicates().values[0]
    temp_df = CLN_data.loc[n]
    for node in temp_df.NodeNo:
        FLENGW = temp_df[temp_df.NodeNo == node].thk.values
        new_CLN.write(
                "%d %d 1 %.2lf %.2lf 1 0 # %s\n" % (n, node, 16000, FLENGW, well_name))

#### Write data for conduits

In [9]:
for c in range(NCONDUITYP):
    new_CLN.write(base_CLN.readline())

## IBOUND and STRT data
 - Open and read ibound and strt files and add lines for new CLN's
 - In ".cln" file, make sure to activate CLN in appropriate Stress Period
 
 We will assume that the wells will become active the same time as for the Hyperion project, Stress Period 61 (15 years into the model in 1986).


#### First, read in existing txt files

In [10]:
#Read in all IBOUND values and set to list to use subsequently for writing new IBOUND file
with open(r"S:/LAX/DUDEK.C001.SMBGSA/INTERA models/UpdatedModel/ModelWithNewExe/input/Arrays/CLN_IBOUND.txt") as f:
    a = f.read().splitlines()
ibound = list()
for i in range(len(a)):
    temp = a[i].split()
    for j in temp:
        ibound.append(j)

In [11]:
base_STRT = open("S:/LAX/DUDEK.C001.SMBGSA/INTERA models/UpdatedModel/ModelWithNewExe/input/Arrays/CLN_StartingHead.txt", 
                  "r")
new_IBOUND = open("S:/LAX/JACOBS.C001.CNSLT/Phase-2/LVL/4000.MODELS/test/CLN_IBOUND_test.IBOUND",
                  "w")
new_STRT = open("S:/LAX/JACOBS.C001.CNSLT/Phase-2/LVL/4000.MODELS/test/CLN_STRT_test.STRT",
                  "w")

In [12]:
#actually writing data to new CLN IBOUND & STRT files
for n in range(base_NCLN):
    new_IBOUND.write(str(ibound[n])+"\n")
    new_STRT.write(base_STRT.readline())
new_STRT.write("\n")
for n in new_CLNs:
    new_IBOUND.write("0\n") #write all new CLN's as inactive to start so they will be activated when we want
    new_STRT.write("%d\n"%new_CLN_data.loc[n,"starting_head"].drop_duplicates().values)

#### Write in CLN file where to read updated starting conditions data

In [13]:
base_CLN.readline() #need to read these two lines to progress
base_CLN.readline()
new_CLN.write("OPEN/CLOSE CLN_IBOUND_test.IBOUND 1 (FREE) -1\n")
new_CLN.write("OPEN/CLOSE CLN_STRT_test.STRT 1 (FREE) -1\n")

42

## Copy lines for each Stress Period the IBOUND & STRT
#### At Stress Period 61, write lines for new CLN's in this scenario

In [14]:
for sp in range(1, 181):
    SPHeader = base_CLN.readline()
    NIB0, NIB1, NIBM1 = int(SPHeader.split()[0]), int(SPHeader.split()[1]), int(SPHeader.split()[2])
    if sp != 61:
        new_CLN.write(SPHeader)
        for j in range(NIB0 + NIB1 + NIBM1):
            new_CLN.write(base_CLN.readline())
    else:
        new_NIB1 = NIB1 + len(new_CLNs)
        new_CLN.write("%d %d %d  #Stress Period %d\n" % (NIB0, new_NIB1, NIBM1, sp))
        for j in range(NIB0 + NIB1 + NIBM1):
            new_CLN.write(base_CLN.readline())
        #new_CLN.write("internal 1 (FREE) 1\n")
        for n in new_CLNs:
            starting_head = new_CLN_data.loc[n,"starting_head"].drop_duplicates().values[0]
            new_CLN.write("%d HEAD %.2lf\n" % (n, starting_head))

            
           

### Close files for writing completion

In [15]:
new_CLN.close()
new_IBOUND.close()
new_STRT.close()